# Antigen Table Condensing
This notebook is to fetch all relevant data for the antigens presented in supplementary table 7 by matching with the 6 prior antigen tables. This is meant to make finding information about antigens (such as their sequence, function, etc...) easier for screening.

In [1]:
import pandas as pd

In [2]:
# let's create a class to represent an antigen
class Antigen:
    def __init__(self):
        # Identifier of the protein
        self.protein_id = None
        
        # Start position of the antigen in the protein
        self.start_position = None
        
        # Sequence of the antigen itself
        self.antigen_sequence = None
        
        # Sequence of the protein that the antigen is a part of
        self.protein_sequence = None 
        
        # Smp protein isoforms
        self.smp_isoform = None
        
        # Smp protein
        self.smp_protein = None
        
        # Subcellular localization of the protein
        self.subcellular_localization = None
        
        # Associated stage
        self.stage = None
        
        # Enrichment information
        self.enrichment = None
        
        # Smp id
        self.smp_id = None
        
        # Function
        self.function = None
        
        # Single cell localization
        self.single_cell_localization = None
        
    # format antigens like they are presented in the tables
    def __str__(self):
        return f'{self.protein_id}|{self.start_position}'
    
    def __repr__(self):
        return f'{self.protein_id}|{self.start_position}'

## Collecting Information from Phage Display Tables

In [3]:
# first, read in the supplementary table 7
df7 = pd.read_excel('Tables/Suppl. Table 7 - Vaccine selected peptides primers and amplification conditions.xlsx', header=1)

In [4]:
# look at the df
df7[:5]

,Code,Unnamed: 1,(Sequence (5'->3'),Product lenght (nt),TM super-fi,Concentration
0,Smp_075800.1|766,Forward primer,CTGACCATGGCATCCAGCTGGATCAAC,208.0,72°C,0.5 uM
1,NaN,Reverse primer,TCATGCTGCAGACGATGGCGGCTAGCG,NaN,NaN,NaN
2,Smp_075800.1|613,Forward primer,CTGACCATGGCATCCAGAATCTTTCTA,208.0,67.9°C,0.5 uM
3,NaN,Reverse primer,TCATGCTGCAGACGATGCTCTTTATACTG,NaN,NaN,NaN
4,Smp_179950.1|307,Forward primer,CTGACCATGGCATCCAGAAATCGAT,208.0,70.2°C,0.5 uM


In [8]:
# fetch all of the relevant antigens from the first column
antigens = {} # map of antigen name to antigen object
for name in df7['Code'].dropna().unique():
    antigen = Antigen()
    antigen.protein_id, antigen.start_position = name.strip().split('|')
    antigens[name.strip()] = antigen
    
# look at the antigens
list(antigens.items())[:5] 

[('Smp_075800.1|766', Smp_075800.1|766),
 ('Smp_075800.1|613', Smp_075800.1|613),
 ('Smp_179950.1|307', Smp_179950.1|307),
 ('Smp_158420.1|307', Smp_158420.1|307),
 ('MEG4.161011091|307', MEG4.161011091|307)]

In [9]:
# now, get the antigen sequences from supplementary table 2
df2 = pd.read_excel('Tables/Suppl. Table 2 - enriched peptides sequences.xlsx', header=1)
df2[:5]

,Peptide.ID,Peptide.sequence,Rhwk08pi,Rhwk10pi,Rhwk12pi,Rhwk1pc,Rhwk4pc,Hmwk12pi,Hmwk22pi
0,c10378-g1-i10123|1,MNLPILGRHSTKLFVFFNCLFITFGIIVIAFGMKLPTKLFRYKRIL...,-,-,-,-,-,-,-
1,c10378-g1-i10123|154,PIFPTATFSGSLSILATCIGILGILNERNMIMYLHLCTLTIVMITE...,-,-,-,-,-,-,-
2,c10378-g1-i10123|307,RNMIMYLHLCTLTIVMITEIGIASSSKLMKDQFFTEAHRSLNMSVQ...,-,-,-,-,-,-,-
3,c10577-g1-i10123|1,TGFKVGINYQPPTVVPGGDLAKVQRAVCMLSNTTAIAEAWARLDHK...,-,-,-,-,-,-,-
4,c10577-g1-i10123|154,WARLDHKFDLMYAKRAFVHWYVGEGMEEGEFSEAREDMAALEKDYE...,-,-,-,-,-,-,-


In [10]:
# extract only rows that match antigens of interest
df2 = df2[df2["Peptide.ID"].isin(antigens.keys())]
df2[:5]

,Peptide.ID,Peptide.sequence,Rhwk08pi,Rhwk10pi,Rhwk12pi,Rhwk1pc,Rhwk4pc,Hmwk12pi,Hmwk22pi
1699,MEG151001110|154,VPAKTTRKHHNTLKTTLHHNTLKTTLCLTTTQYQQKSQHPNTTPSH...,-,yes,yes,yes,yes,-,-
1700,MEG151001110|307,PAKTTRKHHNTLKTTLHHNTLKTTLCLTTTQYQQKSQHPNTTPSHT...,-,yes,yes,yes,yes,-,-
1953,MEG15400100|154,HYVGKLSHHNTVPAKTTRKSQHTTATARHHNTLKTTLSHHNTVPAK...,-,yes,yes,yes,yes,-,-
1968,MEG15410110|307,TTRKSQHTTATARHHNTLKTTLSHHNTVPAKTTRNHTDKTILIKPY...,-,yes,yes,yes,yes,-,-
3255,MEG3.1710010|154,SNTGPQTFYSSTQDDFDKCRNDCGRVTSHHELCGDVCGGNHGGSFP...,-,yes,yes,yes,yes,yes,yes


In [11]:
# now, fill in the antigen sequences
for index, row in df2.iterrows():
    antigen = antigens[row["Peptide.ID"]]
    
    antigen.antigen_sequence = row["Peptide.sequence"]
    enrichment = {}
    for col in ("Rhwk08pi", "Rhwk10pi", "Rhwk12pi", "Rhwk1pc", "Rhwk4pc", "Hmwk12pi", "Hmwk22pi"):
        if row[col] == "yes":
            enrichment[col] = True
        else:
            enrichment[col] = False
    antigen.enrichment = enrichment
            
# peek at the antigens
for antigen in list(antigens.values())[:5]:
    print(antigen, antigen.antigen_sequence, antigen.enrichment)

Smp_075800.1|766 LDQQYKEVKRETNLSHVQRYGDTRMGKLHVSEFQGSRDKSSTENDEPPMKPRHSIASR {'Rhwk08pi': True, 'Rhwk10pi': False, 'Rhwk12pi': False, 'Rhwk1pc': True, 'Rhwk4pc': True, 'Hmwk12pi': False, 'Hmwk22pi': False}
Smp_075800.1|613 NLSIYATTAASPTESSYGTFCDDPTITTCLADLYSYDWIVDSQTHHLTQRTLDQQYKE {'Rhwk08pi': True, 'Rhwk10pi': False, 'Rhwk12pi': False, 'Rhwk1pc': False, 'Rhwk4pc': False, 'Hmwk12pi': False, 'Hmwk22pi': False}
Smp_179950.1|307 KSIATIRDQSRCGSCCAFGAVEAMSERSCIQSGGKQNVELSAVDLEGIVTGSSKENNT {'Rhwk08pi': True, 'Rhwk10pi': False, 'Rhwk12pi': False, 'Rhwk1pc': False, 'Rhwk4pc': False, 'Hmwk12pi': False, 'Hmwk22pi': False}
Smp_158420.1|307 AMSDRSCIQSGGKQNVELSAVDLLSCCEHCGDGFEGGFPALAWDYWVKEGIVTGSSKE {'Rhwk08pi': True, 'Rhwk10pi': False, 'Rhwk12pi': False, 'Rhwk1pc': False, 'Rhwk4pc': True, 'Hmwk12pi': False, 'Hmwk22pi': False}
MEG4.161011091|307 ENKKDKHPTQKEQLPNLQHQRQINDGTSDKTSDTHTIKRQLPNLQHQRTTPKPTTPKT {'Rhwk08pi': True, 'Rhwk10pi': True, 'Rhwk12pi': True, 'Rhwk1pc': True, 'Rhwk4pc': True, 'Hmwk12p

In [12]:
# now, get the protein sequences from the customReferenceProteinsDatabase.fa file
protein_sequences = {}
with open('customReferenceProteinsDatabase.fa') as f:
    for line in f:
        if line.startswith('>'):
            protein_id = line[1:].strip()
            protein_sequences[protein_id] = ''
        else:
            protein_sequences[protein_id] += line.strip()

list(protein_sequences.items())[:5]

[('Smp_127350.1', 'MDGLSTSWTKSSVSDTCQRSNVNIHTMLRNPTYIDRVERR'),
 ('MEG3.1500100',
  'MLFFALILIISLHSFDCVFTARETRDAALSCKKHCEGNNEYVTRYCGGLCSSNTGPQTFYCYLGCSHNASTQDDFDKCRNDCGRVNHGGSFPLCLYNCDQEHPREYERGYDECKCNIS'),
 ('Smp_172500.1',
  'MASLHIHQDHVILVDYRIKYYCMIHPRQLIIIIVLVFICRIHHPQSTKLITTPNMNLTLIQQNRQIRNNSPPPLPPPPKHLIQMLGGDSSIYSTPHGSPQQVNTHKLMILNNSNSNTPNNNNISVDYSPNLLHVSSISKHESSTTGSFRQTDSGVTNMGGSLIDSEQYEPNTIVT'),
 ('MEG8.21201100',
  'MFTIILIYVLYFIANAKFEHTTSGIRNPSKLSDSNASKTLSLKNLTDHYIHTPQKSNNGTSCNGKDTCKLPNPSQKGFTNTTSLPHTQSHNSTVAPSVPKPTRQEIPRSGTIVNGTKPTPGKPVVNGTKPTPGKPESSSIQ'),
 ('Smp_129820.1',
  'MVEYKRRACQPRSQMLFAFTCLIIVCMLPRLYGLLYAGRGWYNQSTFYWLHAATISVYICLAFGELLIQFFSPFNEQNSLLDDKDACPELWASVPALWTFNWLTSTIWKGFRGQINSPSDMFHIRPEDSVNKSYGRFKRAWNRSYNLWFKRKEPETPFSNVMDDDVPLLMDFKENNEDTENVSNDANMLNATPDVDILSPSTTNNNSNSYNLEVCKQSTIKASPARATWGLFYALISSFGLRLFCGWIFISAAVFFNYSSPLLLGMLLRFLSKPEAPLWQGYFIAFSMLFLQSISVLVDQKGFYSCLSLGISVRSALTSAIYRKSLRLSSEARSQYTTGELINLLSVDVNRIKELFMFSFLVWDAFIEFGLSFILLWRQLGSAAIAGVGFLLLVLPLNCLLIWATQ

In [13]:
# update the antigens with the protein sequences
for antigen in antigens.values():
    protein_id = antigen.protein_id
    antigen.protein_sequence = protein_sequences[protein_id]
    
# peek at the antigens
for antigen in list(antigens.values())[:5]:
    print(antigen, antigen.protein_sequence)

Smp_075800.1|766 MMLFSLFLISILHILLVKCQLDTNYEVSDETVSDNNKWAVLVAGSNGYFNYRHQADVCHAYHVLRSKGIKPEHIITMMYDDIAYNLMNPFPGKLFNDYNHKDWYKGVVIDYRGKKVNSKTFLKVLKGDKSAGGKVLKSGKNDDVFIYFTDHGAPGLIAFPDDELYAKEFMSTLKYLHSHKRYSKLVIYIEACESGSMFQRILPSNLSIYATTAASPTESSYGTFCDDPTITTCLADLYSYDWIVDSQTHHLTQRTLDQQYKEVKRETNLSHVQRYGDTRMGKLHVSEFQGSRDKSSTENDEPPMKPRHSIASRDIPLHTLHRQIMMTNNAEDKSFLMQILGLKLKRRDLIEDTMKLIVKVMNNEEIPNTKATIDQTLDCTESVYEQFKSKCFTLQQAPEVGGHFSTLYNYCADGYTAETINEAIIKICG
Smp_075800.1|613 MMLFSLFLISILHILLVKCQLDTNYEVSDETVSDNNKWAVLVAGSNGYFNYRHQADVCHAYHVLRSKGIKPEHIITMMYDDIAYNLMNPFPGKLFNDYNHKDWYKGVVIDYRGKKVNSKTFLKVLKGDKSAGGKVLKSGKNDDVFIYFTDHGAPGLIAFPDDELYAKEFMSTLKYLHSHKRYSKLVIYIEACESGSMFQRILPSNLSIYATTAASPTESSYGTFCDDPTITTCLADLYSYDWIVDSQTHHLTQRTLDQQYKEVKRETNLSHVQRYGDTRMGKLHVSEFQGSRDKSSTENDEPPMKPRHSIASRDIPLHTLHRQIMMTNNAEDKSFLMQILGLKLKRRDLIEDTMKLIVKVMNNEEIPNTKATIDQTLDCTESVYEQFKSKCFTLQQAPEVGGHFSTLYNYCADGYTAETINEAIIKICG
Smp_179950.1|307 MLISVLYIASLISHLEAHISIKNEKFEPLSDDIISYINEHPNAGWRAEKSNRFHSLDDARFQLGARREEPDLRRTRRPTVDHNDWNVEI

In [14]:
# double check that the antigen sequence is always in the protein sequence
for antigen in antigens.values():
    if not antigen.antigen_sequence in antigen.protein_sequence:
        print("WARNING: Antigen sequence not found in protein sequence. An error may have occurred.")
        print(antigen, antigen.antigen_sequence, antigen.protein_sequence)

In [15]:
# now, we can get more information from the remaining supplementary tables
df3 = pd.read_excel('Tables/Suppl. Table 3 - Extracellular proteins enrichment, function and localization.xlsx', header=1)
df3[:5]

,Enriched_peptide ID,Smp_ID (genome V7),Wk08pi,Wk10pi,Wk12pi,Wk1pc,Wk4pc,Hamster wk12+wk22,Function,Single-cell localization
0,Smp_067060.1|460,Smp_067060,yes,--,--,--,--,yes,Cathepsin B-like peptidase (C01 family),Gut
1,Smp_179950.1|307,Smp_179950,yes,--,--,--,--,--,Cathepsin B-like peptidase (C01 family),Gut
2,Smp_075800.1|766,Smp_075800,yes,--,--,yes,yes,--,Asparaginyl endopeptidase,Gut
3,Smp_075800.1|613,Smp_075800,yes,--,--,--,--,--,Asparaginyl endopeptidase,Gut
4,Smp_158420.1|307,Smp_344940,yes,--,--,--,yes,--,cathepsin B peptidase,Gut


In [16]:
# update the antigens with the function and localization

for index, row in df3.iterrows():
    id = row["Enriched_peptide ID"].strip()
    antigen_ids = []
    
    # check if this is a protein or antigen
    if "|" in id: # antigen
        antigen_ids = [id]
    else: # protein
        protein_id = id.split(" ")[0].replace("-","")
        antigen_ids = [str(antigen) for antigen in antigens.values() if protein_id in antigen.protein_id]
    
    # update the antigens
    for antigen_id in antigen_ids:
        # skip any irrelevant antigens
        if antigen_id not in antigens:
            continue
        
        antigen = antigens[antigen_id]
        antigen.smp_id = row["Smp_ID (genome V7)"]
        antigen.function = row["Function"]
        antigen.single_cell_localization = row["Single-cell localization"]
        
# peek at the antigens
for antigen in list(antigens.values())[:5]:
    print(antigen, antigen.smp_id, antigen.function, antigen.single_cell_localization)

Smp_075800.1|766 Smp_075800 Asparaginyl endopeptidase Gut
Smp_075800.1|613 Smp_075800 Asparaginyl endopeptidase Gut
Smp_179950.1|307 Smp_179950 Cathepsin B-like peptidase (C01 family) Gut
Smp_158420.1|307 Smp_344940 cathepsin B peptidase Gut
MEG4.161011091|307 Smp_307220 MEG-4 (10.3) family Oesophageal gland


In [17]:
# now for the subcellular localization and more information
df5 = pd.read_excel('Tables/Suppl. Table 5 - Enriched peptides in each analysis and their localization and stage expression.xls', header=2)
df5[:5]

,Peptide ID,Smp protein \nIsoforms (v7),Smp \nProtein (v7),Subcellular Localization,stage,Hmwk12,Hmwk22,Rhwk08,Rhwk10,Rhwk12,Rhwk43,Rhwk46
0,c11762-g1-i10123|154,NaN,NaN,NaN,NaN,-,yes,-,-,-,-,-
1,c12986-g1-i10123|307,Smp_306120.1,Smp_306120,Cytoplasm,juvenile,-,yes,-,-,-,-,-
2,c1504-g1-i10123|613,Smp_128360.1,Smp_128360,Mitochondrion,adult females,-,yes,-,-,-,-,-
3,c18831-g1-i10123|307,Smp_178360.1,Smp_178360,Cytoplasm|Nucleus,juvenile,-,yes,-,-,-,-,-
4,c19029-g1-i10123|613,Smp_302920.2,Smp_302920,Cytoplasm,juvenile,-,yes,-,yes,yes,yes,yes


In [18]:
# just keep the relevant rows
df5 = df5[df5["Peptide ID"].isin(antigens.keys())]
df5[:5]

,Peptide ID,Smp protein \nIsoforms (v7),Smp \nProtein (v7),Subcellular Localization,stage,Hmwk12,Hmwk22,Rhwk08,Rhwk10,Rhwk12,Rhwk43,Rhwk46
45,MEG151001110|154,Smp_010550.2,Smp_010550,Extracellular,adult females,-,-,-,yes,yes,yes,yes
46,MEG151001110|307,Smp_010550.2,Smp_010550,Extracellular,adult females,-,-,-,yes,yes,yes,yes
78,MEG15400100|154,Smp_010550.2,Smp_010550,Extracellular,adult females,-,-,-,yes,yes,yes,yes
80,MEG15410110|307,Smp_010550.2,Smp_010550,Extracellular,adult females,-,-,-,yes,yes,yes,yes
137,MEG3.1710010|154,Smp_138080.1,Smp_138080,Extracellular,somula,yes,yes,-,yes,yes,yes,yes


In [19]:
# update the antigens
for index, row in df5.iterrows():
    antigen = antigens[row["Peptide ID"]]
    antigen.smp_isoform = row["Smp protein \nIsoforms (v7)"]
    antigen.smp_protein = row["Smp \nProtein (v7)"]
    antigen.subcellular_localization = row["Subcellular Localization"]
    antigen.stage = row["stage"]
    
    for col in ("Hmwk12", "Hmwk22", "Rhwk08", "Rhwk10", "Rhwk12", "Rhwk43", "Rhwk46"):
        if row[col] == "yes":
            antigen.enrichment[col] = True
        else:
            antigen.enrichment[col] = False
            
# peek at the antigens
for antigen in list(antigens.values())[:5]:
    print(antigen, antigen.smp_isoform, antigen.smp_protein, antigen.subcellular_localization, antigen.stage, antigen.enrichment)

Smp_075800.1|766 Smp_075800.1 Smp_075800 Extracellular adult females {'Rhwk08pi': True, 'Rhwk10pi': False, 'Rhwk12pi': False, 'Rhwk1pc': True, 'Rhwk4pc': True, 'Hmwk12pi': False, 'Hmwk22pi': False, 'Hmwk12': False, 'Hmwk22': False, 'Rhwk08': True, 'Rhwk10': False, 'Rhwk12': False, 'Rhwk43': True, 'Rhwk46': True}
Smp_075800.1|613 Smp_075800.1 Smp_075800 Extracellular adult females {'Rhwk08pi': True, 'Rhwk10pi': False, 'Rhwk12pi': False, 'Rhwk1pc': False, 'Rhwk4pc': False, 'Hmwk12pi': False, 'Hmwk22pi': False, 'Hmwk12': False, 'Hmwk22': False, 'Rhwk08': True, 'Rhwk10': False, 'Rhwk12': False, 'Rhwk43': False, 'Rhwk46': False}
Smp_179950.1|307 Smp_067060.1 Smp_067060 Extracellular adult females {'Rhwk08pi': True, 'Rhwk10pi': False, 'Rhwk12pi': False, 'Rhwk1pc': False, 'Rhwk4pc': False, 'Hmwk12pi': False, 'Hmwk22pi': False, 'Hmwk12': False, 'Hmwk22': False, 'Rhwk08': True, 'Rhwk10': False, 'Rhwk12': False, 'Rhwk43': False, 'Rhwk46': False}
Smp_158420.1|307 Smp_344940.1 Smp_344940 Extracell

In [20]:
# now, let's write this all out to a new table
with open('antigens.csv', 'w') as f:
    f.write('Protein ID,Start Position,Antigen Sequence,Protein Sequence,Smp Isoform,Smp Protein,Subcellular Localization,Stage,Function,Single Cell Localization\n')
    for antigen in antigens.values():
        f.write(f'{antigen.protein_id},{antigen.start_position},{antigen.antigen_sequence},{antigen.protein_sequence},{antigen.smp_isoform},{antigen.smp_protein},{antigen.subcellular_localization},{antigen.stage},{antigen.function},{antigen.single_cell_localization}\n')

## BLASTing Antigens and Proteins

In [18]:
%pip install biopython
from Bio.Blast import NCBIWWW
import os

Note: you may need to restart the kernel to use updated packages.


In [19]:
# Ensure the output directory exists
os.makedirs('blast_results/antigens', exist_ok=True)

# let's blast the antigen sequences
def blast_antigen(antigen):
    print(f'BLASTing antigen {antigen}')
    result_handle = NCBIWWW.qblast("blastp", "nr", antigen.antigen_sequence)
    with open(f'blast_results/antigens/{antigen}.xml', 'w') as f:
        f.write(result_handle.read())
    result_handle.close()
    print(f'BLASTing antigen {antigen} done')
    

for antigen in antigens.values():
    # check if this antigen has been blasted already
    if os.path.exists(f'blast_results/antigens/{antigen}.xml'):
        print(f'Antigen {antigen} already BLASTed')
        continue
    
    blast_antigen(antigen)

Antigen Smp_075800.1|766 already BLASTed
Antigen Smp_075800.1|613 already BLASTed
Antigen Smp_179950.1|307 already BLASTed
Antigen Smp_158420.1|307 already BLASTed
Antigen MEG4.161011091|307 already BLASTed
Antigen MEG4.1610110|154 already BLASTed
Antigen MEG4.11101100|154 already BLASTed
Antigen MEG4.1700100|154 already BLASTed
Antigen MEG4.1200001|307 already BLASTed
Antigen MEG4.11711010172|154 already BLASTed
Antigen MEG4.1700110|154 already BLASTed
Antigen MEG4.11601101172|154 already BLASTed
Antigen MEG4.11710010172|154 already BLASTed
Antigen MEG4.1901110|307 already BLASTed
Antigen MEG4.1610100|154 already BLASTed
Antigen MEG4.1711010|154 already BLASTed
Antigen MEG4.1701100|307 already BLASTed
Antigen MEG4.11500110|154 already BLASTed
Antigen MEG8.2400100|154 already BLASTed
Antigen MEG8.21500100|154 already BLASTed
Antigen MEG8.21501010|154 already BLASTed
Antigen MEG8.21510010181|154 already BLASTed
Antigen MEG8.21401000|154 already BLASTed
Antigen MEG8.21200110|154 already 

In [20]:
# Ensure the output directory exists
os.makedirs('blast_results/proteins', exist_ok=True)

# find all protein sequences
proteins = set()
for antigen in antigens.values():
    proteins.add((antigen.protein_id, antigen.protein_sequence))
    
# let's blast the protein sequences
def blast_protein(id, seq):
    print(f'BLASTing protein {id}')
    result_handle = NCBIWWW.qblast("blastp", "nr", seq)
    with open(f'blast_results/proteins/{id}.xml', 'w') as f:
        f.write(result_handle.read())
    result_handle.close()
    print(f'BLASTing protein {id} done')
    

for id, seq in proteins:
    # check if this antigen has been blasted already
    if os.path.exists(f'blast_results/proteins/{id}.xml'):
        print(f'Protein {id} already BLASTed')
        continue
    
    blast_protein(id, seq)

Protein MEG8.21200100 already BLASTed
Protein MEG4.11601101172 already BLASTed
Protein MEG4.1711010 already BLASTed
Protein MEG4.11710010172 already BLASTed
Protein MEG8.2400100 already BLASTed
Protein MEG8.2901100 already BLASTed
Protein Smp_179950.1 already BLASTed
Protein MEG4.1901110 already BLASTed
Protein MEG4.11500110 already BLASTed
Protein MEG8.21510100 already BLASTed
Protein MEG8.21510010181 already BLASTed
Protein MEG8.21401000 already BLASTed
Protein MEG4.1610100 already BLASTed
Protein MEG3.1710010 already BLASTed
Protein MEG4.1700110 already BLASTed
Protein MEG4.1610110 already BLASTed
Protein MEG15410110 already BLASTed
Protein MEG15400100 already BLASTed
Protein MEG3.1711000 already BLASTed
Protein MEG4.1700100 already BLASTed
Protein MEG8.2511000 already BLASTed
Protein MEG151001110 already BLASTed
Protein MEG4.1200001 already BLASTed
Protein MEG4.1701100 already BLASTed
Protein Smp_075800.1 already BLASTed
Protein MEG4.11101100 already BLASTed
Protein Smp_158420.1 al

## AlphaFold Predictions